Import Libraries

In [1]:
import pandas as pd
import datetime
from datetime import datetime, timedelta
import os
import polars as pl
import gc
import numpy as np

import s3fs
import boto3
from io import BytesIO as bo

Calculate Date Variables

In [2]:
#variables -
this_day = datetime.today()
# ### FOR TESTING - REMOVE LATER ### 
this_day = this_day - timedelta(days=7)
# ####
days_to_monday = (this_day.weekday() - 0) % 7
monday = this_day - timedelta(days=days_to_monday)

CUR_PROC_WK = monday.strftime("%Y%m%d")

PRE_PROC_WK0 = monday - timedelta(days=7)
PRE_PROC_WK = str(PRE_PROC_WK0.year) + str(PRE_PROC_WK0.month).zfill(2) + str(PRE_PROC_WK0.day).zfill(2)

CUR_WK0 = monday - timedelta(days=17)
CUR_WK = str(CUR_WK0.year) + str(CUR_WK0.month).zfill(2) + str(CUR_WK0.day).zfill(2)

PRE_WK0 = monday - timedelta(days=24)
PRE_WK = PRE_WK0.strftime("%Y%m%d")


In [3]:
bucket = 'vortex-staging-a65ced90'

Library Names and File Paths :

In [4]:
raw_path = f'PYADM/raw/{CUR_PROC_WK}/inbound/'
curwk = f'PYADM/raw/{CUR_PROC_WK}/dataframes/'
prewk = f'PYADM/raw/{PRE_PROC_WK}/dataframes/'
wkxpn = 'PYADM/weekly/staging/xponent/'
mthxpn = 'PYADM/monthly/staging/xponent/'
pwkxpn = f'PYADM/weekly/archive/{PRE_WK}/xponent/'

In [5]:
#Picking Up Data from Rx_1
curwk_RX_PERIOD_WKSUM_CUR_WK = pl.read_parquet(f's3://{bucket}/{curwk}RX_PERIOD_WKSUM_CUR_WK.parquet')

In [6]:
temp_WK_LAX = curwk_RX_PERIOD_WKSUM_CUR_WK.filter(pl.col("MKT_CD")=="LAX")

#temp_WK_LAX = temp_WK_LAX.with_columns(pl.col('ProductID').map_elements(lambda x: f"{x:08}",return_dtype = pl.datatypes.Utf8) 
#This approach was slower by 2 seconds

def pad_with_zeros(val):
    return str(val).zfill(8)

temp_WK_LAX = temp_WK_LAX.with_columns(pl.col('ProductID').map_elements(pad_with_zeros,return_dtype=pl.String))
del curwk_RX_PERIOD_WKSUM_CUR_WK
gc.collect()

0

In [7]:
temp_WK_LAX = temp_WK_LAX.drop(['ProductName','MKT_CD'])

rename_items = {
    'IronwoodID':'IID','ProductID':'PG_ID',
    'TotalRx':'TRX','NewRx':'NRX',
    'TotalQuantity':'TUN','NewQuantity':'NUN',
    'TotalFactoredQuantity':'TUF','NewFactoredQuantity':'NUF'
}

temp_WK_LAX = temp_WK_LAX.rename(rename_items)

In [8]:
std_sec_def = pl.read_parquet(f's3://{bucket}/PYADM/reference/std_sec_def.parquet')

temp_WK_TMP_LAX = pl.DataFrame()
temp_WK_TMP_LAX = temp_WK_LAX.join(std_sec_def,on='PG_ID',how='left')

del temp_WK_LAX
gc.collect()

0

In [9]:
#THIS MAY OR MAY NOT CRASH, CONSIDER REMOVING isNULL EXTRA CONDITION
temp_WK_TMP1_LAX = temp_WK_TMP_LAX.filter((pl.col('PROD_CD')!='LIN') | (pl.col('PROD_CD').is_null()))
del temp_WK_TMP_LAX
gc.collect() #NEED TO EXPORT THIS !!!

0

In [10]:
#temp_WK_TMP1_LAX.write_parquet(curwk+'\\temp_WK_TMP1_LAX.parquet',compression='zstd',compression_level=10,use_pyarrow=True)

temp_WK_TMP1_LAX.to_pandas().to_parquet(f's3://{bucket}/{curwk}temp_WK_TMP1_LAX.parquet',compression='snappy')

In [11]:
## FOR QC DELETE LATER
#print(temp_WK_TMP_LAX.shape[0]-temp_WK_TMP1_LAX.shape[0])
# print(set(temp_WK_TMP_LAX['PROD_CD'].unique())-set(temp_WK_TMP1_LAX['PROD_CD'].unique()))
# print(len(temp_WK_TMP1_LAX['PROD_CD'].unique()))

In [12]:
## NOTE-
# Only passing through ChannelID 1 and 2 , lax_n for weekly and monthly are summerized 
# using temp_WK_TMP1_LAX
# Note that NWAY MISSING is used in the summary step , i.e missing values are considered for aggreation

In [13]:
#In the creation of lax_n only channel 1 and 2 are kept , but we dont really get any other info than that , this step 
# might be redundant
fil_temp_WK_TMP1_LAX = temp_WK_TMP1_LAX.filter((temp_WK_TMP1_LAX['ChannelID'] == '1') | (temp_WK_TMP1_LAX['ChannelID'] == '2'))
del temp_WK_TMP1_LAX

In [14]:
# Did this to check which columns had null values , will be replacing them by "" while summary
# group_cols = ['IID','WK_END_DATE','CCYYMM','MKT_CD','MarketName','G_B','RO_TYPE','PFAM_CD','PFAM_NAME','PROD_NAME']
# for col_name in group_cols:
#     has_null = fil_temp_WK_TMP1_LAX[col_name].is_null().sum() > 0
#     print(col_name,fil_temp_WK_TMP1_LAX[col_name].dtype,has_null)


# NOTE : ALL NULL TYPE VALUES WILL BE REPLACED BY "" , DID NOT CODE IN COLUMN SPECIFIC RULES 

In [15]:
group_cols = ['IID','WK_END_DATE','MKT_CD','MarketName','G_B','RO_TYPE','PFAM_CD','PFAM_NAME','PROD_CD','PROD_NAME']
sum_cols = ['TRX','NRX','TUN','NUN','TUF','NUF']

# #perform groupby and calc function 
wkxpn_LAX_N = pl.DataFrame()
wkxpn_LAX_N = fil_temp_WK_TMP1_LAX.fill_null("").group_by(group_cols).agg(
    [pl.col(c).sum() for c in sum_cols]
)


In [16]:
group_cols = ['IID','CCYYMM','MKT_CD','MarketName','G_B','RO_TYPE','PFAM_CD','PFAM_NAME','PROD_CD','PROD_NAME']
sum_cols = ['TRX','NRX','TUN','NUN','TUF','NUF']

# #perform groupby and calc function 
mthxpn_LAX_N = pl.DataFrame()
mthxpn_LAX_N = fil_temp_WK_TMP1_LAX.fill_null("").group_by(group_cols).agg(
    [pl.col(c).sum() for c in sum_cols]
)


In [19]:
# END OF 6th Code
# Weekly and Monthly Level LAX_N datasets are ready 
#[Note They only Contain 105 Weeks worth of data]
#Ask Team if Data is required to be Exported.

In [18]:
wkxpn_LAX_N.to_pandas().to_parquet(f's3://{bucket}/{curwk}wkxpn_LAX_N.parquet',compression='snappy')
mthxpn_LAX_N.to_pandas().to_parquet(f's3://{bucket}/{curwk}mthxpn_LAX_N.parquet',compression='snappy')